# SubstitutesDept Drycker - Coop Case Study - Complementary and Substitute Products 

##  Substitute Products Products PyFIM

## Preprocessing

In [6]:
# Libraries 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pylab as pylab
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from sklearn.metrics import mean_squared_error as mse 
from sklearn.neural_network import MLPRegressor
from apyori import apriori ## Importing apriori package
from fim import arules
#from mlxtend.frequent_patterns import apriori, association_rules

params = {'legend.fontsize': 'x-large',
          'figure.figsize': (22, 14),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}

pylab.rcParams.update(params)

#### Import functions from ./Functions_Forecasting AR file in this same folder, look for comments on the functions in that notebook



In [7]:
#from ipynb.fs.defs.Functions_Forecasting import * 


### importing data

In [9]:
data = pd.read_csv("../4months_v2/rl_4months.csv")

In [10]:
data.shape

(3221229, 40)

In [11]:
#data[["receiptKey","DayDate","hourOfDay","minuteOfHour","quantity","ItemID"]].iloc[0:20,]

In [12]:
# Generate unique name 
data['Unique_Item_Name'] = data['ItemCategoryName'] + '_' + data['ItemSegmentName']

In [13]:
data_subset = data[['receiptKey', 'DayDate','hourOfDay', 'ItemID', 'quantity','Unique_Item_Name', 'CoopOnlineYN', 'ItemCategoryTeamName']]



### Deletion of returns and false duplicates of Negative and 0 Quantities

In [14]:
data_subset = data_subset[data_subset.quantity>0]

### Aggregating Datasets

In [15]:
data_subset = data_subset[data_subset['CoopOnlineYN'] == 'Y']

In [16]:
data_subset_sub = data_subset[data_subset['ItemCategoryTeamName'] == 'DRYCKER']

In [17]:
data_subset_sub = data_subset_sub[['receiptKey', 'ItemID']]


In [18]:
transactions = [a[1]['ItemID'].tolist() for a in list(data_subset_sub.groupby(['receiptKey']))] ## Combing all the items in list format for each cutomer

In [19]:
hash_map_series = data_subset.groupby(['ItemID']).first()['Unique_Item_Name']

In [20]:
hash_map_series = pd.DataFrame(hash_map_series.str.strip())# Remove 

In [21]:
hash_map_series

,Unique_Item_Name
ItemID,
2922,FIKABRÖD_0
10914,KROPPSVÅRD_KVINNA
11501,FÄRDIGMAT & FISK_KÖTTKONSERVER
11503,GODIS_STYCKSAKER
11541,GODIS_STYCKSAKER
...,...
751980,GRÖNSAKER_0
779855,GRÖNSAKER_0
781191,FRUKT & BÄR_0


In [22]:
transactions

[[300223],
 [294172],
 [637686],
 [494344, 42001],
 [300067, 275860, 616537],
 [643049, 495201, 518549],
 [300221],
 [294256],
 [294258, 294259, 274864],
 [616546],
 [145185, 494344],
 [294337, 616537],
 [145269],
 [294263, 294095],
 [287162],
 [495124],
 [80070, 288415, 287405],
 [288225],
 [300229, 616546],
 [300067, 300069],
 [300075, 635518],
 [300098, 300223, 97804],
 [300082],
 [287384, 294095],
 [288478],
 [288454, 288484],
 [288502],
 [287370],
 [287913],
 [145357, 300082],
 [123563, 123592, 300083],
 [300082],
 [13030, 288200],
 [637686, 287385, 326523],
 [139737, 294099, 294333, 139736],
 [300082, 436408, 97772],
 [288402, 649521],
 [612446],
 [300082, 145269],
 [145121],
 [630647, 294395, 494859],
 [300224],
 [123571, 288110, 24963, 294263],
 [287391, 300224, 287389, 287388],
 [612443, 145269, 300072],
 [288206, 287460, 288202],
 [287271],
 [288225],
 [145354, 300082],
 [294256, 630650],
 [288173, 300223, 559668, 520277],
 [300075, 276831],
 [88254, 145356, 288200, 294065],


## PyFIM

In [23]:
# inputs
supp = 2 # minimum support of an assoc. rule (default: 10)
conf = 10 # minimum confidence of an assoc. rule (default: 80%)
report = 'asC'

In [24]:
# make dict for nicer looking column names
report_colnames = {
    'a': 'support_itemset_absolute',
    's': 'support_itemset_relative',
    'S': 'support_itemset_relative_pct',
    'b': 'support_bodyset_absolute',
    'x': 'support_bodyset_relative',
    'X': 'support_bodyset_relative_pct',
    'h': 'support_headitem_absolute',
    'y': 'support_headitem_relative',
    'Y': 'support_headitem_relative_pct',
    'c': 'confidence',
    'C': 'confidence_pct',
    'l': 'lift',
    'L': 'lift_pct',
    'e': 'evaluation',
    'E': 'evaluation_pct',
    'Q': 'xx',
    'S': 'support_emptyset',
    }

In [25]:
result = arules(transactions, supp=supp, conf=conf, report=report)

In [26]:
colnames = ['consequent', 'antecedent'] + [report_colnames.get(k, k) for k in list(report)]
df_rules = pd.DataFrame(result, columns=colnames)
df_rules = df_rules.sort_values('support_itemset_absolute', ascending=False)
print(df_rules.shape)


(7, 5)


In [27]:
hash_map_series

,Unique_Item_Name
ItemID,
2922,FIKABRÖD_0
10914,KROPPSVÅRD_KVINNA
11501,FÄRDIGMAT & FISK_KÖTTKONSERVER
11503,GODIS_STYCKSAKER
11541,GODIS_STYCKSAKER
...,...
751980,GRÖNSAKER_0
779855,GRÖNSAKER_0
781191,FRUKT & BÄR_0


In [28]:
results_df_names = df_rules.merge(hash_map_series, left_on='consequent', right_on='ItemID').rename(columns= {'Unique_Item_Name': 'consequent names'})


In [29]:
results_df_names['antecedent_names'] = results_df_names.antecedent.apply(lambda x: [hash_map_series.loc[i][0] for i in x])

In [30]:
results_df_names

,consequent,antecedent,support_itemset_absolute,support_itemset_relative,confidence_pct,consequent names,antecedent_names
0,300082,(),1018,0.101556,10.155626,KAFFE TE CHOKLAD_BRYGGKAFFE,[]
1,300082,"(494859,)",53,0.005287,20.229008,KAFFE TE CHOKLAD_BRYGGKAFFE,[KAFFE TE CHOKLAD_KAFFETILLBEHÖR]
2,300082,"(97804,)",46,0.004589,18.699187,KAFFE TE CHOKLAD_BRYGGKAFFE,[KAFFE TE CHOKLAD_KAFFETILLBEHÖR]
3,300082,"(145269,)",39,0.003891,10.025707,KAFFE TE CHOKLAD_BRYGGKAFFE,[LÄSK&SAFT_MAINSTREAM COLA LIGH]
4,300082,"(145121,)",33,0.003292,11.262799,KAFFE TE CHOKLAD_BRYGGKAFFE,[LÄSK&SAFT_MAINSTREAM COLA]
5,145276,"(145269,)",39,0.003891,10.025707,LÄSK&SAFT_MAINSTREAM APELSIN L,[LÄSK&SAFT_MAINSTREAM COLA LIGH]
6,145269,"(145121,)",38,0.003791,12.969283,LÄSK&SAFT_MAINSTREAM COLA LIGH,[LÄSK&SAFT_MAINSTREAM COLA]


# Apriori Algorithm PyFIM Libary

In [31]:
from fim import eclat

In [50]:
# inputs
supp = 2 # minimum support of an assoc. rule (default: 10)
conf = 10 # minimum confidence of an assoc. rule (default: 80%)
report = 'asC'

In [51]:
result = eclat(transactions, supp=supp, conf=conf, report=report)

In [52]:
# make dict for nicer looking column names
report_colnames = {
    'a': 'support_itemset_absolute',
    's': 'support_itemset_relative',
    'S': 'support_itemset_relative_pct',
    'b': 'support_bodyset_absolute',
    'x': 'support_bodyset_relative',
    'X': 'support_bodyset_relative_pct',
    'h': 'support_headitem_absolute',
    'y': 'support_headitem_relative',
    'Y': 'support_headitem_relative_pct',
    'c': 'confidence',
    'C': 'confidence_pct',
    'l': 'lift',
    'L': 'lift_pct',
    'e': 'evaluation',
    'E': 'evaluation_pct',
    'Q': 'xx',
    'S': 'support_emptyset',
    }

In [53]:
result = arules(transactions, supp=supp, conf=conf, report=report)

In [54]:
colnames = ['consequent', 'antecedent'] + [report_colnames.get(k, k) for k in list(report)]
df_rules = pd.DataFrame(result, columns=colnames)
df_rules = df_rules.sort_values('support_itemset_absolute', ascending=False)
print(df_rules.shape)


(7, 5)


In [55]:
results_df_names = df_rules.merge(hash_map_series, left_on='consequent', right_on='ItemID').rename(columns= {'Unique_Item_Name': 'consequent names'})


In [56]:
results_df_names = results_df_names.dropna()

In [57]:
results_df_names['antecedent_names'] = results_df_names.antecedent.apply(lambda x:"" if x== () else [hash_map_series.loc[i][0] for i in x])

In [58]:
results_df_names 

,consequent,antecedent,support_itemset_absolute,support_itemset_relative,confidence_pct,consequent names,antecedent_names
0,300082,(),1018,0.101556,10.155626,KAFFE TE CHOKLAD_BRYGGKAFFE,
1,300082,"(494859,)",53,0.005287,20.229008,KAFFE TE CHOKLAD_BRYGGKAFFE,[KAFFE TE CHOKLAD_KAFFETILLBEHÖR]
2,300082,"(97804,)",46,0.004589,18.699187,KAFFE TE CHOKLAD_BRYGGKAFFE,[KAFFE TE CHOKLAD_KAFFETILLBEHÖR]
3,300082,"(145269,)",39,0.003891,10.025707,KAFFE TE CHOKLAD_BRYGGKAFFE,[LÄSK&SAFT_MAINSTREAM COLA LIGH]
4,300082,"(145121,)",33,0.003292,11.262799,KAFFE TE CHOKLAD_BRYGGKAFFE,[LÄSK&SAFT_MAINSTREAM COLA]
5,145276,"(145269,)",39,0.003891,10.025707,LÄSK&SAFT_MAINSTREAM APELSIN L,[LÄSK&SAFT_MAINSTREAM COLA LIGH]
6,145269,"(145121,)",38,0.003791,12.969283,LÄSK&SAFT_MAINSTREAM COLA LIGH,[LÄSK&SAFT_MAINSTREAM COLA]
